In [1]:
import glob
import os
import numpy as np

import cv2
import pandas as pd

import src.config as config

In [2]:
dfs = {}
files = glob.glob(os.path.join(config.LABELS_PATH, "*.tsv"))
for file_path in files:
    path, file_name = os.path.split(file_path)
    id, ext = os.path.splitext(file_name)
    dfs[id] = pd.read_csv(file_path, sep="\t")

In [12]:
import mediapipe as mp


mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose


def mask_image(image: cv2.Mat):
    # mask using fillpoly
    mask = np.zeros(image.shape[:2], dtype=np.uint8)
    a = 300
    b = 100
    mask = cv2.fillPoly(
        mask,
        [
            np.array([[a, 0], [b, image.shape[0]], [image.shape[1] - b, image.shape[0]], [image.shape[1] - a, 0],])
        ],
        255,
    )
    return cv2.bitwise_and(image, image, mask=mask)


def get_pose_instance():
    return mp_pose.Pose(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5,
        model_complexity=0,
        static_image_mode=False,
        smooth_landmarks=False
    )


with get_pose_instance() as pose1, get_pose_instance() as pose2:
    for id, df in dfs.items():
        video_path = os.path.join(config.VIDEOS_PATH, f"{id}.mp4")
        capture = cv2.VideoCapture(video_path)
        for index, row in df.iterrows():
            start_frame = row["start"]
            end_frame = row["end"]
            capture.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
            step = 1
            for _ in range(start_frame, end_frame, step):
                for _ in range(step):
                    _, frame = capture.read()
                # To improve performance, optionally mark the image as not writeable to
                # pass by reference.
                image = frame.copy()
                image = mask_image(image)
                image.flags.writeable = False
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

                h, w = image.shape[:2]
                image1 = image[: h // 2 + 20, 200 : w - 200]
                results = pose1.process(image1)
                image1.flags.writeable = True
                image1 = cv2.cvtColor(image1, cv2.COLOR_RGB2BGR)
                # mp_drawing.plot_landmarks(results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS)
                mp_drawing.draw_landmarks(
                    image1,
                    results.pose_landmarks,
                    mp_pose.POSE_CONNECTIONS,
                    landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style(),
                )

                image2 = image[h // 2 - 120 :, 100 : w - 100]
                results = pose2.process(image2)
                image2.flags.writeable = True
                image2 = cv2.cvtColor(image2, cv2.COLOR_RGB2BGR)
                mp_drawing.draw_landmarks(
                    image2,
                    results.pose_landmarks,
                    mp_pose.POSE_CONNECTIONS,
                    landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style(),
                )
                
                image[: h // 2 + 20, 200 : w - 200] = image1
                image[h // 2 - 120 :, 100 : w - 100] = image2
                cv2.imshow('result', image)

                cv2.waitKey(1)
        else:
            cv2.destroyAllWindows()